# RL Trading Model Training & Evaluation

This notebook demonstrates how to train a reinforcement learning (RL) trading model using your full feature pipeline and visualize the results. It loads historical price data, extracts features, trains a PPO agent, and evaluates performance.

In [1]:
# ================================================
# 🔧 SETUP - Add src to Python Path
# ================================================

import sys
import os

# Add src directory to Python path so 'core' module can be found
project_root = os.getcwd()
src_path = os.path.join(project_root, 'src')

if src_path not in sys.path:
    sys.path.insert(0, src_path)
    print(f"✅ Added to Python path: {src_path}")
else:
    print(f"✅ Already in path: {src_path}")

# Verify
print(f"📂 Working directory: {project_root}")
print(f"🔍 Python will search for modules in: {src_path}")
print("=" * 50)

✅ Added to Python path: d:\Dev\trading-bot\src
📂 Working directory: d:\Dev\trading-bot
🔍 Python will search for modules in: d:\Dev\trading-bot\src


In [ ]:
# Section 6: Complete Model Test Report with Updated Charts
print("📋 Generating Complete Test Report with Updated Interactive Charts...")
print("=" * 70)

# Import our clean testing solution
from src.reporting.model_test_report import quick_test_model

# Run model test - charts will display directly in notebook
print("🚀 Running model test with improved chart design...")

quick_test_model(
    model_dir='models/rl_optimized',
    symbol='BTCUSDT', 
    test_candles=5000
)

📋 Generating Complete Test Report with Updated Interactive Charts...
🚀 Running model test with improved chart design...
🔍 Testing model: models/rl_optimized
📊 Symbol: BTCUSDT, Candles: 1000
📊 Loading data...
📥 Loading data for BTCUSDT...
 - Adding indicators...
 - Dropping NaNs for...
Data loading complete.
🤖 Loading model and generating predictions...
✅ GPU Available: NVIDIA GeForce RTX 3080 (10.0GB)
🖥️ RL Training Device: cuda
🎯 Generating predictions for 1000 candles...
Dropped 2 rows containing NaNs
TradingEnvironment: Aligned df and indicator_features by removing first 2 rows.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
✅ Model loaded on cuda
✅ Test completed! Shape: (901, 16)
📊 Columns: ['price', 'reward', 'initial_balance', 'step', 'signal', 'position_size', 'entry_price', 'position_shares', 'cash', 'cash_used', 'unrealized_pnl', 'equity', 'step_pnl', 'traded', 'total_trades', 'is_bankrupt']

📊 SIMPLE PERFORMANCE SUMMARY
💰 Initial Balance: $100,0

,price,reward,initial_balance,step,signal,position_size,entry_price,position_shares,cash,cash_used,unrealized_pnl,equity,step_pnl,traded,total_trades,is_bankrupt
271274,122208.78,0.0,100000.0,96,1,0.0,0.0,0.0,100000.00,0.0,0.0,100000.00,0.0,False,0,False
271275,122229.00,0.0,100000.0,97,1,0.0,0.0,0.0,100000.00,0.0,0.0,100000.00,0.0,False,0,False
271276,122292.43,0.0,100000.0,98,1,0.0,0.0,0.0,100000.00,0.0,0.0,100000.00,0.0,False,0,False
271277,122231.40,0.0,100000.0,99,1,0.0,0.0,0.0,100000.00,0.0,0.0,100000.00,0.0,False,0,False
271278,122129.53,0.0,100000.0,100,1,0.0,0.0,0.0,100000.00,0.0,0.0,100000.00,0.0,False,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272170,115040.55,0.0,100000.0,992,1,0.0,0.0,0.0,99589.17,0.0,0.0,99589.17,0.0,False,134,False
272171,114751.33,0.0,100000.0,993,1,0.0,0.0,0.0,99589.17,0.0,0.0,99589.17,0.0,False,134,False
272172,115258.40,0.0,100000.0,994,1,0.0,0.0,0.0,99589.17,0.0,0.0,99589.17,0.0,False,134,False
272173,115205.07,0.0,100000.0,995,1,0.0,0.0,0.0,99589.17,0.0,0.0,99589.17,0.0,False,134,False
